# Présentation de la boîte à outil Julia MaxPlus.jl pour l'algèbre (max,+)

## Algèbre (max,+)

L'algèbre (max,+) (se prononce max-plus) redéfinit les opérateurs addition et multiplication de l'algèbre classique par respectivement les opérateurs maximum noté $\oplus$ et addition noté $\otimes$ dans le domaine des nombres réels $\mathbb{R}$ augmenté du nombre moins l'infini ($\varepsilon = -\infty$) que l'on nomme $\mathbb{R}_{\varepsilon} = \mathbb{R} \cup \{ -\infty \}$. Sa structure algébrique est celle d'un dioïde selectif-inversible selon la classification de Gondran-Minoux (cette structure est appelée plus fréquemment semi-corps idempotent) $(\mathbb{R}_{\varepsilon}, \oplus, \otimes)$.

$$\forall a,b \in \mathbb{R}_{\varepsilon}: a \oplus b \triangleq \max(a,b)$$
$$\forall a,b \in \mathbb{R}_{\varepsilon}: a \otimes b \triangleq a + b$$

L'intérêt du calcul matriciel dans cette algèbre est enseigné dès les années 60 par J. Kuntzman dans sa théorie des réseaux. Elle est utilisée dans de nombreux
domaines Recherche opérationnelle (théorie des réseaux), Physique (Quantification), Probabilité (transformée de Cramer), Automatique (systèmes à événements discrets), Informatique (théorie des automates, Réseaux de Pétri), Mathématiques (géométrie algébrique).

Dans un précédent tuotriel, nous vous avons présenté comment installer notre boîte à outils Max-Plus pour le langage Julia dont le but essentiel est de faciliter les calculs matriciels dans cette algèbre. Elle peut être téléchargée depuis GitHub à l'adresse https://github.com/Lecrapouille/MaxPlus.jl ou bien depuis le système de paquets de Julia via la commande `] add MaxPlus`. Elle est un portage dans Julia de l'arithmétique Max-Plus de [ScicosLab](http://www.scicoslab.org), un fork de Scilab en cours de remplacement par le logiciel [NSP](https://cermics.enpc.fr/~jpc/nsp-tiddly/mine.html).

Ce présent document présente uniquement les fonctions de base de la boîte à outils tout en introduisant les bases de l'algèbre (max,+). L'algèbre (min,+) fera l'objet d'un autre tutoriel. On pourra consulter la [bibliographie](../docs/src/bibliography.md) pour obtenir la démonstration de certains résultats et la description des algorithmes utilisés. Pour ceux qui désirent comparer cette boîte à outils avec Sicoslab, on rappelle qu'un nombre Max-Plus Sicoslab est créé par la fonction maxplus abrégée par `#`, que les éléments neutres sont notés `%0` et `%1`, qu'une aide et une démonstration sont accessibles depuis les menus de ScicosLab. Enfin le document PDF [suivant](https://jpquadrat.github.io/TPALGLIN.pdf) décrit les fonctionnalités Max-plus de ScicosLab.

## Démarrer la boîte à outils Max-Plus pour Julia

Depuis le REPL de Julia, lancez Jupyter notebook :

In [1]:
# using IJulia
# notebook()

Dans un document Jupyter nouvellement créé, chargez la boîte à outil Max-Plus depuis le dossier MaxPlus.jl:

In [2]:
push!(LOAD_PATH, pwd())
using MaxPlus

┌ Info: Precompiling MaxPlus [41177cfe-c387-11e9-2806-edd030e4594e]
└ @ Base loading.jl:1662
┌ Warning: Replacing docs for `MaxPlus.MP :: Tuple{AbstractVector, AbstractVector, AbstractVector}` in module `MaxPlus`
└ @ Base.Docs docs/Docs.jl:240


Pour le moment, dans un soucis pédagogique, on active un mode d'affichage particulier des nombres Max-Plus affichant explicitement $-\infty$ et les $0.0$. Des modes d'affichage plus agréables seront expliqués par la suite.

In [3]:
set_tropical_display(0)

I will show -Inf and 0.0

Cette boîte à outils permet de générer du code $\LaTeX$ via la fonction `Base.show`. Dans Jupyter ce mode semble être celui utilisé par défault, mais ici, on veut aussi garder l'affichage en texte plein. Pour cela on doit d'abord taper:

In [4]:
Base.show(io::IO, ::MIME"text/latex", x::MP) = show(io, MIME"text/plain", x)
Base.show(io::IO, ::MIME"text/latex", A::MPAbstractVecOrMat) = show(io, MIME"text/plain", A)

## Scalaires Max-Plus

Avant de présenter l'algèbre (max,+), créons quelques scalaires Max-Plus grâce au constructeur `MP()`:

In [5]:
a = MP(1.0);  b = MP(3.5);  c = MP(5)
typeof(a), typeof(b), typeof(c)

(MP, MP, MP)

Dans Scilab on aurait écrit `#(1.0)`. Dans Julia, les nombres Max-Plus sont codés en interne par des `Float64` car ils sont définis dans l'espace $\mathbb{R}_{\varepsilon}$. Pour plus de souplesse le constructeur `MP()` accepte des entiers: ils sont alors converti `Float64`. Pour repasser dans l'algébre classique `(+,*)` on peut directement accéder à la valeur via le champ `λ` des objets Julia.

In [6]:
a,   a.λ,   typeof(a),   typeof(a.λ)

(1.0, 1.0, MP, Float64)

In [7]:
c,   c.λ,   typeof(c),   typeof(c.λ)

(5.0, 5.0, MP, Float64)

Si l'on ne desire pas utiliser explicitement le champ `λ` la fonction `plustimes` le fait aussi (son nom vient du `+` et du `*` de l'algèbre classique). Elle fonctionne également pour les matrices creuses et pleines :

In [8]:
a,   plustimes(a),   typeof(a),   typeof(plustimes(a))

(1.0, 1.0, MP, Float64)

In [9]:
a + a,   plustimes(a) + plustimes(a)

(1.0, 2.0)

`plustimes` est une fonction Scilab mais des méthodes avec des noms plus dans l'esprit de Julia sont aussi possibles:

In [10]:
Float64(a)

1.0

In [11]:
float(a)

1.0

Les nombres Max-Plus contaminent les autres nombres (entiers, réels) : ils convertissent un nombre non Max-Plus en nombre Max-Plus via les opérateurs arithmétiques où les opérateurs de promotion implicites. On effet, d'une part on considère que l'on travaille uniquement dans cette algébre et d'autre part cela simplifie l'écriture du code (ainsis que sa lecture) :

In [12]:
d = 1.0
typeof(d),   typeof(c),   typeof(c + d),   typeof((c + d).λ),   c + d

(Float64, MP, MP, Float64, 5.0)

Nous voyons que l'addition Max-Plus a converti `d` de type `Float64` en type `MP`. Même comportement pour les nombres entiers où `f` de type `Int64` est converti en en type `MP` :

In [13]:
f = 1
typeof(f), typeof(c), typeof(c + f), typeof((c + f).λ),   c + f

(Int64, MP, MP, Float64, 5.0)

## Les constantes Max-Plus pour Julia

Les éléments neutres pour les opérateurs $\oplus$ et $\otimes$ sont donnés sous forme de constantes Julia :
- Elément neutre $\varepsilon$ (parfois noté $\mathbb{0}$ dans certains documents et sur Scilab `%0`) pour l'opérateur $\oplus$ : les constantes Julia sont `mp0` ou bien `ε` (obtenu en tapant `\varepsilon`) valant $-\infty$. Cet élément est absorbant pour la multiplication
$\varepsilon\otimes a=a\otimes \varepsilon=\varepsilon$.
- Elément neutre $e$ (parfois noté $\mathbb{1}$ dans certains documents et sur Scilab `%1`) pour l'opérateur $\otimes$ : les constants Julia sont
`mp1` ou bien `mpe` valant 0.
- Bien que cette boîte à outil est dédiée à l'alègbre Max-Plus elle utilise la constante `mptop` valant $+\infty$ lorsqu'elle fait des calculs dans l'algèbre duale Min-Plus. Cette constante correspond au `%top` de ScicosLab.

Ces nombres sont de type `MP` (et l'on pourra éventuellement les convertir en nombre de l'algèbre classique soit via le champ `λ` soit la fonction `plustimes`.

In [14]:
mp0, ε, mp1, mpe, mptop, zero(MP), one(MP), mptop

(-Inf, -Inf, 0.0, 0.0, Inf, -Inf, 0.0, Inf)

## Contrôle de l'affichage des nombres Max-Plus

Nous voyons que jusqu'à présent, Julia affiche dans ses résultats `-Inf` et `0.0` pour `ε` et `0` ce qui n'est pas très compact. En Max-Plus on manipule souvent de grosse matrices et un bon affichage est important. C'est parce qu'en début de document nous avons écrit `set_tropical_display(0)` pour forcer cet affichage en `mode 0` dans un soucis pédagogique afin que le lecteur ne confonde pas les zéros Max-Plus des zéros de l'algèbre classique, mais il existe quatre styles possibles d'affichage des nombres Max-Plus que l'on peut commuter avec la fonction `set_tropical_display` qui accepte un nombre entre 0 et 4. Le mode `1` étant celui défini par défaut et suit l'affichage dans ScicosLab car il permet d'afficher les matrices de façon compact. En effet, il est commun en Max-Plus de devoir manipuler et afficher de grosses matrices remplies d'éléments $\varepsilon$.

- Style 0 : est l'affichage classique de Julia: les nombres $-\infty$ sont affichés `-Inf` et les nombres sous forme de réels comme `0.0`.
- Style 1 ou 2 : les nombres $-\infty$ sont affiché sous la forme d'un point `.`.
- Style 3 ou 4 : les nombres $-\infty$ sont affichés sous la forme `ε`.
- Style 1 ou 3 : les nombres réels qui peuvent être écrits comme des entiers (donc sans chiffres apès la virgule) seront affichés comme des entiers. Par exemple `0`.
- Style 2 ou 4 : les zéros sont affichés `e`.

Le style activé impacte les fonctions `Base.show` et impact également la fonction `LaTeX` pour la génération de code $\LaTeX$ pour les matrices (que l'on verra un peu plus tard dans ce document).

In [15]:
# Affichage classique façon Julia
set_tropical_display(0)

J = MP([-Inf 0; 0 -Inf])

I will show -Inf and 0.0

2×2 (max,+) dense matrix:
  -Inf    0.0
   0.0   -Inf


In [16]:
# Affichage des 0 sous la forme de e
set_tropical_display(2)

J

I will show (max,+) -Inf and (min,+) +Inf as . and 0.0 as e

2×2 (max,+) dense matrix:
  .   e
  e   .


In [17]:
# Affichage des -Inf sous la forme de ε
set_tropical_display(3)

J

I will show (max,+) -Inf and (min,+) +Inf as ε

2×2 (max,+) dense matrix:
  ε   0
  0   ε


In [18]:
# Affichage des -Inf sous la forme de ε et les 0 sous la forme de e
set_tropical_display(4)

J

I will show (max,+) -Inf and (min,+) +Inf as ε and 0.0 as e

2×2 (max,+) dense matrix:
  ε   e
  e   ε


Et finalement, le mode par défaut :

In [19]:
# Affichage des -Inf sous la forme d'un .
set_tropical_display(1)

J

I will show (max,+) -Inf and (min,+) +Inf as .

2×2 (max,+) dense matrix:
  .   0
  0   .


## Opérateur Max-Plus $\oplus$

L'opérateur addition est redéfini par l'opérateur `max` de l'algèbre classique. Son symbole, pour le différencier de l'addition dans l'algèbre classique, est $\oplus$. Mais en Julia on gardera le symbole `+`. Cet opérateur est associatif, commutatif, a un élément neutre (noté $\varepsilon$) et est idempotent. $\forall a,b,c \in \mathbb{R}_{\varepsilon}:$

$$a \oplus b \triangleq \max(a,b)$$

In [20]:
a = MP(1); b = MP(3); c = MP(5);
(a, b, c)

(1, 3, 5)

In [21]:
a + b   # ≜ max(a, b) = max(1, 3) = 3

(max,+) 3

####  $\oplus$ n'est pas inversible ni simplifiable

L'égalité suivante $a \oplus b = a \oplus c$ n'entraine pas $b = c$. Par contre on aura $a \oplus b = a$ si $a \geq b$ ou plus généralement $a \oplus b = a$ ou $b$. Selon la terminologie de Gondran-Minoux $\oplus$ est sélectif.

#### Commutativité de $\oplus$

$$a \oplus b = b \oplus a$$
$$\triangleq$$
$$\max(a,b) = \max(b,a)$$

In [22]:
a + b == b + a

true

#### Associativité de $\oplus$

$$a \oplus b \oplus c = (a \oplus b) \oplus c = a \oplus (b \oplus c)$$

In [23]:
a + b + c == (a + b) + c == a + (b + c)

true

In [24]:
a + b + c # ≜ max(a, b, c) == max(1, 3, 5)

(max,+) 5

#### Elément neutre $\varepsilon$ pour $\oplus$

$$a \oplus \varepsilon = \varepsilon \oplus a = a$$
$$\triangleq$$
$$\max(a,-\infty) = \max(-\infty,a) = a$$

In [25]:
a + ε == ε + a == a

true

Equivallent à :

In [26]:
a + mp0 == mp0 + a == a

true

In [27]:
(a, mp0, ε), (a + mp0, a + ε), (mp0 + a, ε + a)

((1, ., .), (1, 1), (1, 1))

Notons que `0` est neutre pour les nombres positifs :

In [28]:
a + 0 == 0 + a == a

true

In [29]:
a, 0, a + 0

(1, 0, 1)

#### $\oplus$ est idempotent

In [30]:
a + a    # ≜ max(a, a) == max(1, 1) == 1

(max,+) 1

## Opérateur Max-Plus $\otimes$

L'opérateur multiplication est redéfini par l'opérateur addition qui est associatif, commutatif, a l'élément neutre $e$, l'élément absorbant $\varepsilon$ et est distributif sur $\oplus$.

In [31]:
a * b    # ≜ a + b == 1 + 3 == 4

(max,+) 4

#### Commutativité de $\otimes$

$$a \otimes b = b \otimes a$$
$$\triangleq$$
$$a + b = b + a$$

In [32]:
a * b == b * a

true

#### Associativité de $\otimes$

$$a \otimes b \otimes c = (a \otimes b) \otimes c = a \otimes (b \otimes c)$$

In [33]:
a * b * c == (a * b) * c == a * (b * c)

true

In [34]:
a * b * c

(max,+) 9

#### Elément neutre $e$ pour $\otimes$

$$a \otimes e = e \otimes a = a$$
$$\triangleq$$
$$a + 0 = 0 + a = a$$

In [35]:
a * mpe == mpe * a == a

true

Equivalent à :

In [36]:
a * mp1 == mp1 * a == a

true

#### Elément absorbant $\varepsilon$ pour $\otimes$

$$a \otimes \varepsilon = \varepsilon \otimes a = \varepsilon$$
$$\triangleq$$
$$a -\infty = -\infty + a = -\infty$$

In [37]:
a * ε == ε * a == ε

true

Equivalent à :

In [38]:
a * mp0 == mp0 * a == mp0

true

Par convention:

$$+\infty \otimes \varepsilon = \varepsilon \otimes +\infty = \varepsilon$$

In [39]:
mptop * mp0 # FIXME shall return mp0

(max,+) .

#### $\otimes$ n'est pas idempotente

In [40]:
a * a    # ≜ a + a == 1 + 1 == 2

(max,+) 2

### Distributivité de $\otimes$ sur $\oplus$

$$a \otimes (b \oplus c) = (a \otimes b) \oplus (a \otimes c)$$


In [41]:
(a + b) * c == (a * c) + (b * c)     # => max(a, b) + c == max(a + c, b + c) 

true

In [42]:
(a * c) + (b * c)

(max,+) 8

### Opérateur puissance

En algèbre Max-Plus l'opérateur puissance se comporte comme une multiplication dans l'algèbre classique :

In [43]:
MP(2)^5   # ==> 2 * 5

(max,+) 10

In [44]:
MP(2)^0   # ==> 2 * 0

(max,+) 0

In [45]:
MP(2)^-1   # ==> 2 * -1

(max,+) -2

Au lieu de `^-1` on peut aussi appeler la fonction `inv()` :

In [46]:
inv(MP(2))

(max,+) -2

## Vecteur colonnes Max-Plus, matrices Max-Plus (dense et creux)

Ce que l'on vient de voir pour les sclaires est également applicable aux vecteurs, matrices qu'ils soient denses (pleines) ou creuses. 

### Construction de vecteurs colonnes denses Max-Plus

In [47]:
MP(1:5)

5-element (max,+) vector:
  1
  2
  3
  4
  5


In [48]:
MP(1:0.5:3)

5-element (max,+) vector:
    1
  1.5
    2
  2.5
    3


In [49]:
MP([1, 2, 3])

3-element (max,+) vector:
  1
  2
  3


### Construction de matrices denses Max-Plus

Comme pour les sclaires, la contamination des nombres Max-Plus sur les nombres Int64 et Float64 fonctionne également sur les éléments des matrices denses et creuses :

In [50]:
[MP(1) 2; 3.5 4.0]

2×2 (max,+) dense matrix:
    1   2
  3.5   4


`MP(1)` de type `MP` a contaminé les nombres entiers *classiques* `2`, `3.0` et `4` en nombre `MP`.

Voici une autre façon plus élégante de l'écrire :

In [51]:
MP([1 2; 3 4])

2×2 (max,+) dense matrix:
  1   2
  3   4


Autre exemple de contamination des nombres Max-Plus:

In [52]:
f = 3; a = MP(1)
[f       a
 f + f   a + a]

2×2 (max,+) dense matrix:
  3   1
  6   1


`f + f` étant des `Int64` l'addition classique a été faite avant la promotion en nombre `MP`. Par contre, `a + a` étant des `MP` c'est l'addition (max, +) qui a été utilisée. Finallement tous les éléments de la matrices sont de type `MP`.

Dans l'exemple suivant $\varepsilon$ a rendu la matrice suivante implicitement Max-Plus :

In [53]:
[ε 2; 3.5 4]

2×2 (max,+) dense matrix:
    .   2
  3.5   4


La contamination fonctionne également sur les matrices creuses.

### Construction de matrices creuses Max-Plus

Une matrice creuse est une matrice contenant beaucoup de zéros. Sa structure interne est conçue pour ne pas garder en mémoire ces zéros (sauf en Julia s'ils sont explicitement donnés). En algèbre classique les zéros sont 0 (pour les entiers) ou 0.0 (réels) mais en Max-Plus ils ont pour valeur $-\infty$ et par conséquent une matrice creuse Max-Plus ne stocke pas les $\varepsilon$. 

Les matrices creuses sont essentielles pour les applications qui necessitent souvent des matrices de grandes tailles. Par exemple dans le calcul du chemin le plus long dans
un reseau routier, la taille de la matrice sera le nombre de noeuds, le nombre d'éléments
non nuls (le nombre de routes joignant 2 noeuds) va croitre linéairement avec cette taille, alors que le nombre de coefficients de la matrice croit comme le carré de la taille.

Pour créer une matrice creuse Max-Plus, plusieurs choix:
- soit à partir d'une matrice creuse initialement vide, comme la fonction `mpzeros` que l'on verra plus tard.
- soit à partir d'une matrice pleine avec la fonction `SparseArrays.sparse` couplée avec le constructeur `MP`. 
- soit à partir de trois vecteurs et la fonction `MP` : un vecteur des données à stocker et deux vecteurs indiquant les index de ces données dans la matrice.

Dans tous les cas il est important d'importer le bon module `using SparseArrays`.

In [54]:
using SparseArrays;
set_tropical_display(1);

I will show (max,+) -Inf and (min,+) +Inf as .

#### A partir d'une matrice pleine

In [55]:
S = MP(sparse([1 2; 0 4]))

2×2 (max,+) sparse matrix with 3 stored entries:
  [1, 1]  =  1
  [1, 2]  =  2
  [2, 2]  =  4

Ici le zéro de l'algèbre classique (valant 0) a été supprimé par `SparseArrays.sparse` mais dans l'exemple suivant c'est le zéro de l'algébre Max-Plus ($\varepsilon$ vallant $-\infty$) qui a sera supprimé.

In [56]:
S = sparse(MP([1 2; -Inf 4]))

2×2 (max,+) sparse matrix with 3 stored entries:
  [1, 1]  =  1
  [1, 2]  =  2
  [2, 2]  =  4

Le lecteur attentif aura compris que l'affichage est celui de Julia 1.5 même si Julia >= 1.6 est utilisé. En effet, avec Julia 1.6 l'affichage d'une matrice creuse se fait de la même manière qu'une matrice dense. L'ancien affichage est forcé mais uniquement pour les matrices creuses Max-Plus.

Pour rappel, la fonction `SparseArrays.findnz` retourne les éléments stockés `D` ainsi que leur indices `I` et `J` sous forme d'un triplet de vecteurs lignes ce qui devient rapidement illisible dès que la matrice grandit un peu :

In [57]:
i,j,d = findnz(S)

([1, 1, 2], [1, 2, 2], MP[1, 2, 4])

#### Construction explicite creuse

Tout comme `SparseArrays.findnz` retournant un triplet de vecteur colonnes `I`, `J` et `D`, la `SparseArrays.sparse` accepte ses mêmes paramètres. Mais les zéros explicites seront stockés :

In [58]:
S = MP(sparse([1; 2; 3], [1; 2; 3], [42; 0; 5]))

3×3 (max,+) sparse matrix with 3 stored entries:
  [1, 1]  =  42
  [2, 2]  =  0
  [3, 3]  =  5

Ici le zéro de l'algèbre classique (vallant 0) n'a pas été supprimé par `SparseArrays.sparse`. Dans l'exemple suivant c'est le zéro de l'algébre Max-Plus ($\varepsilon$ vallant $-\infty$) qui n'a pas été supprimé.

In [59]:
S = sparse([1; 2; 3], [1; 2; 3], MP([42; -Inf; 5]))

3×3 (max,+) sparse matrix with 3 stored entries:
  [1, 1]  =  42
  [2, 2]  =  .
  [3, 3]  =  5

Ceci est un comportement voulu par Julia pour plus de souplesse car vous pouvez appeller `dropzeros` pour les supprimer.

In [60]:
dropzeros(S)

3×3 (max,+) sparse matrix with 2 stored entries:
  [1, 1]  =  42
  [3, 3]  =  5

Pour cela, la constructeur `MP(I,J,Tv)` a été ajoouté: il appelle `dropzeros(sparse(I,J,Tv))` afin de supprimer les zéros explicites :

In [61]:
S = MP([1; 2; 3], [1; 2; 3], MP([0; -Inf; 5]))

3×3 (max,+) sparse matrix with 2 stored entries:
  [1, 1]  =  0
  [3, 3]  =  5

### Conversion de matrices Max-Plus vers algèbre classique

Comme vu précédement pour les scalaires, on peut vouloir convertir une matrice Max-Plus en matrice classique :

In [62]:
A = MP([4 0; 7 -Inf])
plustimes(A)

2×2 Matrix{Float64}:
 4.0    0.0
 7.0  -Inf

Fonctionne aussi pour les matrices creuses :

In [63]:
Z = spzeros(MP,2,2)

2×2 (max,+) sparse matrix with 0 stored entries

In [64]:
plustimes(Z)

2×2 SparseMatrixCSC{Float64, Int64} with 4 stored entries:
 -Inf  -Inf
 -Inf  -Inf

On peut vouloir passer d'une matrice creuse Max-Plus en matrice pleine  dans l'algèbre classique :

In [65]:
Matrix(plustimes(Z))

2×2 Matrix{Float64}:
 -Inf  -Inf
 -Inf  -Inf

### Conversion d'une matrice creuse en matrice pleine :

Les trois fonctions produisent le même résultat :

In [66]:
full(Z),  dense(Z),   Matrix(Z)

(MP[. .; . .], MP[. .; . .], MP[. .; . .])

### Construction de vecteurs colonnes creux Max-Plus

Tout comme les matrices creuses plusieurs façons de faire. En préservant les zero():

In [67]:
sparsevec([1, 3], MP([0, -Inf]))

3-element SparseVector{MP, Int64} with 2 stored entries:
  [1]  =  0
  [3]  =  .

In [68]:
MP(sparsevec([1, 3], [0, -Inf]))

3-element SparseVector{MP, Int64} with 2 stored entries:
  [1]  =  0
  [3]  =  .

Où en supprimant les zeros:

In [69]:
MP([1, 3], [0, -Inf])

3-element SparseVector{MP, Int64} with 1 stored entry:
  [1]  =  0

## Construction de matrices Max-Plus usuelles

### Matrice dense d'identité

Par exemple de taille 2 $\times$ 2 :

$$\left[
\begin{array}{*{20}c}
e & \varepsilon \\
\varepsilon & e \\
\end{array}
\right]$$

Depuis Julia v1.0, la fonction `eye` n'existe plus et a été remplacée par `LinearAlgebra.I` mais cette boite à outils ajoute leur équivalent `eye(MP,...)` et `mpI` :

In [70]:
using LinearAlgebra
I

UniformScaling{Bool}
true*I

In [71]:
Matrix{MP}(I, 2, 2)

2×2 (max,+) dense matrix:
  0   .
  .   0


In [72]:
mpI

UniformScaling{MP}
0*I

In [73]:
Matrix(mpI, 2, 2)

2×2 (max,+) dense matrix:
  0   .
  .   0


In [74]:
Matrix(mpI, 2, 2) == eye(MP,2,2),   Matrix{MP}(I, 2, 2) == eye(MP,2,2)

(true, true)

La fonction `mpeye` reste plus simple à taper :

In [75]:
J = eye(MP,2,2)

2×2 (max,+) dense matrix:
  0   .
  .   0


In [76]:
J = eye(MP,2) # Equivalent à eye(MP, 2,2)

2×2 (max,+) dense matrix:
  0   .
  .   0


Taille 3 $\times$ 2 :

In [77]:
J = eye(MP,3,2)

3×2 (max,+) dense matrix:
  0   .
  .   0
  .   .


Construire une matrice identitée (max,+) à partir des dimensions d'une matrice (max,+) existante :

In [78]:
A = MP([1.0 -Inf; 0.0 4])
J = eye(A)

2×2 (max,+) dense matrix:
  0   .
  .   0


### Matrices/Vecteur colonne denses remplies uniquement de $e$ :

Par exemple matrice de taille 2 $\times$ 2 :

$$\left[
\begin{array}{*{20}c}
e & e \\
e & e \\
\end{array}
\right]$$

In [79]:
O = ones(MP,2,2)

2×2 (max,+) dense matrix:
  0   0
  0   0


Vecteur colonne de 2 éléments:

In [80]:
O = ones(MP,2) # /!\ N'est pas équivalent à ones(MP,2,2) /!\

2-element (max,+) vector:
  0
  0


Construire une matrice de e (max,+) à partir des dimensions d'une matrice (max,+) existante :

In [81]:
A = MP([1.0 -Inf; 0.0 4])
J = ones(A)

2×2 (max,+) dense matrix:
  0   0
  0   0


### Matrices creuses remplies de $\varepsilon$ :

$$\left[
\begin{array}{*{20}c}
\varepsilon & \varepsilon \\
\varepsilon & \varepsilon \\
\end{array}
\right]$$

**Attention :** Sous Scilab `zeros` va créer une matrice creuse alors que nativement Julia va créer une matrice pleine, il faudra bien penser à appeller `spzeros` à la place pour avoir le même comportement.

#### Matrice dense

In [82]:
Z = zeros(MP,2,2)

2×2 (max,+) dense matrix:
  .   .
  .   .


In [83]:
Z = zeros(MP,2,3)

2×3 (max,+) dense matrix:
  .   .   .
  .   .   .


In [84]:
Z = zeros(MP,2)

2-element (max,+) vector:
  .
  .


#### Matrice creuse

In [85]:
Z = spzeros(MP,2,2)

2×2 (max,+) sparse matrix with 0 stored entries

In [86]:
Z = spzeros(MP,2,3)

2×3 (max,+) sparse matrix with 0 stored entries

In [87]:
Z = spzeros(MP,2)

2-element SparseVector{MP, Int64} with 0 stored entries

On remarquera que ces matrices sont vides. En effet, elles correspondent aux 0 éliminés des matrices creuses en algèbre classique. Une matrice creuse Max-Plus ne stocke pas les nombres Max-Plus $-\infty$ (**note:** enfin jusqu'à Julia > 1.9 car les versions précédentes avaient un bogue elles confondaient 0 et zero(T) avec T template de type MP).

### Matrices denses remplies de $\varepsilon$ :

Il faut utiliser la fonction `full` ou bien la fonction synonyme `dense`.

In [88]:
Z = full(zeros(MP,2,2))

2×2 (max,+) dense matrix:
  .   .
  .   .


### Matrices diagonales

Dense:

In [89]:
diagm(MP([1,2,3]))

3×3 (max,+) dense matrix:
  1   .   .
  .   2   .
  .   .   3


Creuse:

In [90]:
spdiagm(MP([1,2,3]))

3×3 (max,+) sparse matrix with 3 stored entries:
  [1, 1]  =  1
  [2, 2]  =  2
  [3, 3]  =  3

## Opérateur éléments par éléments sur les matrices

Julia permet d'itérer sur les éléments d'un tableau, matrice, vecteur et appliquer une opération sur chacun d'eux. Par exemple :

$$4 \oplus \left[
\begin{array}{*{20}c}
2 \\
8\\
\end{array}
\right] = \left[
\begin{array}{*{20}c}
4 \oplus 2 \\
4 \oplus 8\\
\end{array}
\right] = \left[
\begin{array}{*{20}c}
4 \\
8\\
\end{array}
\right]$$

In [91]:
A = MP([1.0 2; 3 4])

2×2 (max,+) dense matrix:
  1   2
  3   4


On applique la fonction max(2, ) sur chacun des éléments qui seront contaminés en nombre Max-Plus :

In [92]:
2 .+ A

2×2 (max,+) dense matrix:
  2   2
  3   4


In [93]:
A .+ 2.0

2×2 (max,+) dense matrix:
  2   2
  3   4


On applique la fonction +(2, ) sur chacun des éléments 

In [94]:
2 .* A

2×2 (max,+) dense matrix:
  3   4
  5   6


In [95]:
A .* 2.0

2×2 (max,+) dense matrix:
  3   4
  5   6


## Addition et produit matriciel

Les matrices peuvent être de type Max-Plus. L'addition et le produit matriciel Max-Plus correspond à l'addition et au produit matriciel avec les opérateurs $+$ et $\times$ surchargés.

### Addition matricielle

$$\begin{bmatrix}
1 & 6 \\
8 & 3
\end{bmatrix} \oplus \begin{bmatrix}
2 & 5 \\
3 & 3
\end{bmatrix} = \begin{bmatrix}
1 \oplus 2 & 6 \oplus 5 \\
8 \oplus 3 & 3 \oplus 3
\end{bmatrix} = \begin{bmatrix}
2 & 6 \\
8 & 3
\end{bmatrix}$$

In [96]:
MP([1 6; 8 3]) + MP([2 5; 3 3])

2×2 (max,+) dense matrix:
  2   6
  8   3


### Produit matriciel

$$A=\begin{bmatrix}
4 & 3 \\
7 & -\infty
\end{bmatrix}\;,$$

$$A \otimes A = \begin{bmatrix}
4 \otimes 4 \oplus 3 \otimes7 & 4 \otimes 3 \oplus 3 \otimes -\infty \\
7 \otimes 4 \oplus -\infty \otimes 7 & 7 \otimes 3 \oplus -\infty \otimes -\infty
\end{bmatrix}\; = \begin{bmatrix}
10 & 7 \\
11 & 10
\end{bmatrix}\; = A^2.$$

In [97]:
A = MP([4 3; 7 -Inf])
A * A

2×2 (max,+) dense matrix:
  10    7
  11   10


In [98]:
A * A == A^2

true

Fonctionne également sur les matrices creuses :

In [99]:
A = MP([4 3; 7 -Inf])
sparse(A)

2×2 (max,+) sparse matrix with 3 stored entries:
  [1, 1]  =  4
  [2, 1]  =  7
  [1, 2]  =  3

In [100]:
A * sparse(A) == sparse(A) * A == sparse(A) * sparse(A)

true

Puissance d'une matrice Max-Pus :

In [101]:
A^5

2×2 (max,+) dense matrix:
  24   23
  27   24


In [102]:
A^0

2×2 (max,+) dense matrix:
  0   .
  .   0


S'applique également aux matrices rectangulaires compatibles :

In [103]:
MP([2 0; mp0 5]) * MP([2; 8])

2-element (max,+) vector:
   8
  13


In [104]:
MP([2 8]) * MP([2 0; mp0 5])

1×2 (max,+) dense matrix:
  4   13


Vérifions que la matrice identité $I$ est bien neutre :

$$ A \otimes I = I \otimes A = A$$

In [105]:
A = MP([4 3; 7 -Inf])
A * eye(MP, 2,2) == eye(MP, 2,2) * A == A

true

Vérifions la matrice zéro est bien absorbante :

In [106]:
A * zeros(MP, 2,2) == zeros(MP, 2,2) * A == zeros(MP, 2,2)

true

In [107]:
A + zeros(MP, 2,2) == zeros(MP, 2,2) + A == A

true

## Affichage des matrices Max-Plus en LaTeX

A partir d'une matrice Max-Plus, on peut générer le code $\LaTeX$ grâce à la fonction `LaTeX` ou via la fonction `show` avec l'argument `MIME"text/latex"`. La fonction `set_tropical_display` modifie en conséquence les éléments neutres et absorbants du code LaTeX généré.

In [108]:
set_tropical_display(0)

I will show -Inf and 0.0

In [109]:
LaTeX(stdout, eye(MP, 2,2))

"\\left[\n\\begin{array}{*{20}c}\n0 & -\\infty \\\\\n-\\infty & 0 \\\\\n\\end{array}\n\\right]\n"

Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
0 & -\infty \\
-\infty & 0 \\
\end{array}
\right]$$

Alors que :

In [110]:
set_tropical_display(2)

I will show (max,+) -Inf and (min,+) +Inf as . and 0.0 as e

In [111]:
LaTeX(stdout, eye(MP, 2,2))

"\\left[\n\\begin{array}{*{20}c}\ne & . \\\\\n. & e \\\\\n\\end{array}\n\\right]\n"

Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
e & . \\
. & e \\
\end{array}
\right]$$

Fonctionne avec les matrices creuses :

In [112]:
set_tropical_display(1)

I will show (max,+) -Inf and (min,+) +Inf as .

In [113]:
LaTeX(stdout, zeros(MP, 2,2))

"\\left[\n\\begin{array}{*{20}c}\n. & . \\\\\n. & . \\\\\n\\end{array}\n\\right]\n"

Une fois ce code $\LaTeX$ compilé, il affichera :

$$\left[
\begin{array}{*{20}c}
. & . \\
. & . \\
\end{array}
\right]$$

## Compatibilité avec les autres paquets Julia

Vous pouvez utiliser les autres paquets Julia tel que `using LinearAlgebra` ou et certaines fonctions devraient fonctionner directement avec Max-Plus. Certaines ont été corrigée.

## Trace d'une matrice Max-Plus

La trace est la somme Max-Plus des éléments diagonaux.

In [114]:
A = MP([4 3; 7 -Inf])
tr(A)

(max,+) 4

In [115]:
tr(A) == A[1,1] + A[2,2]

true

In [116]:
A = [5 mp0 5; mp0 6 3; 11 12 11]

3×3 (max,+) dense matrix:
   5    .    5
   .    6    3
  11   12   11


In [117]:
tr(A)

(max,+) 11

In [118]:
tr(A) == A[1,1] + A[2,2] + A[3,3]

true

## Norme d'une matrice

Elle est définie comme la différence entre le plus grand et le moins grand coéfficient de la matrice.

In [119]:
norm(A)

(max,+) Inf

In [120]:
norm(eye(MP, 2,2))

(max,+) Inf

### Autres exemples

In [121]:
diagm(MP([1,2,3]))

3×3 (max,+) dense matrix:
  1   .   .
  .   2   .
  .   .   3


## Résiduation d'une matrice Max-Plus

Elle permet de calculer la plus grande sous solution de $Ax\leq b$.

### Valeurs propres d'une matrice Max-Plus $A v = \lambda v$

Le spectre d'une matrice est l'ensemble de ses valeurs propres. Une matrice carrée $A \in \mathbb{R}_{\varepsilon}^{n \times n}$ a une valeur propre s'il existe un nombre réel $\lambda \in \mathbb{R}^{n}$ et un vecteur $v \in \mathbb{R}_{\varepsilon}^{n}$ si :

$$A v = \lambda v$$

$v$ est appelé vecteur propre et $\lambda$ valeur propre.

In [122]:
S = MP([3 7; 2 4])
λ,v = howard(sparse(S))

(MP[4, 4], MP[7, 4])

In [123]:
# FIXME KO
S * v == λ[1] * v

false

In [124]:
# FIXME KO
S * v == λ[2] * v

false

Ces éléments spectraux donnent le comportement asymptotique des systèmes dynamiques linéares Max-Plus $x_n=A\otimes x_{n-1}$ qui croissent linéairement à une vitesse donnée par la valeur propre qui est unique dès que le graphe d'adjacence de la matrice est fortement connexe :

In [125]:
x = MP([1; 0])

2-element (max,+) vector:
  1
  0


In [126]:
[x S*x S^2*x S^3*x]

2×4 (max,+) dense matrix:
  1   7   11   16
  0   4    9   13


Dans le boite à outils Max-Plus de Scilab, ils sont obtenus soit par les fonctions `karp` ou `howard`. Dans cette boite à outils Julia seul `howard` est donné car son algorithme est plus rapide (linéaire au nombre d'arcs) que l'algorithme de `karp` ($O(mn)$).

In [127]:
# FIXME KO
S = MP(sprand(10000,10000,0.0005))+0.001*sparse(eye(MP, 10000,10000))
@time λ,v = howard(S)

  5.970418 seconds (75 allocations: 11.177 GiB, 4.20% gc time)


(MP[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.8284451906212126, 0.001, 0.001  …  0.8284451906212126, 0.7896814048913782, 0.7896814048913782, 0.7896814048913782, 0.001, 0.001, 0.7896814048913782, 0.7896814048913782, 0.8284451906212126, 0.001], MP[110.16362977758328, 13.487745539361587, 91.63443916838918, 82.74600288814901, 64.27216065597037, 25.522108670134603, 95.66368940559144, 1.4509428597601761, 21.129797083940968, 2.7362456909599793  …  0.4082716469634122, 1.386253376578908, 2.2393177356673526, 1.404178338653879, 29.184760024544047, 5.722439427333541, 1.4884581504661716, 1.6135948132919355, 0.7515089868533087, 34.97202948212506])

### Résolution d'équations linéaires Max-Plus $A \otimes x = b$

Soit $A \in \mathbb{R}_{\varepsilon}^{n \times n}$ une matrice Max-plus carrée et $b \in \mathbb{R}_{\varepsilon}^{n}$ un vecteur colonne. La solution de $A \otimes x = b$ est donnée par :
$$x = A^{-1} \otimes b$$

L'inverse de $A$ peut être donné par les fonctions Julia `inv` ou `^-1` et si la matrice n'est pas inversible alors une erreur Julia est levée :

In [128]:
A = [mp0 1 mp0; 2 mp0 mp0; mp0 mp0 3]

3×3 (max,+) dense matrix:
  .   1   .
  2   .   .
  .   .   3


In [129]:
inv(A)   # Ou bien A^-1

3×3 (max,+) dense matrix:
   .   -2    .
  -1    .    .
   .    .   -3


Vérifions que leur produit donne bien la matrice identité :

In [130]:
A * inv(A)   # Ou bien inv(A) * A

3×3 (max,+) dense matrix:
  0   .   .
  .   0   .
  .   .   0


Calculons $x = A^{-1} \otimes b$ :

In [131]:
B = [3 mp0 mp0; mp0 mp0 4; mp0 5 mp0]

3×3 (max,+) dense matrix:
  3   .   .
  .   .   4
  .   5   .


In [132]:
x = A \ B

3×3 (max,+) dense matrix:
  .   .   2
  2   .   .
  .   2   .


In [133]:
A * x == B

true

### Résolution d'équations linéaires Max-Plus $x = Ax \oplus b$

Soit $A \in \mathbb{R}_{\varepsilon}^{n \times n}$ une matrice Max-plus carrée et $b \in \mathbb{R}_{\varepsilon}^{n}$ un vecteur colonne. La solution de $x = Ax \oplus b$ est donnée par :
$$x = A^* b$$

Où :

$$A^+ \triangleq A^1 \oplus A^2 \oplus A^3 \oplus\;...$$
$$A^* \triangleq A^0 \oplus A^+$$

$A^0$ n'est d'autre que la matrice identité Max-Plus. $A^+$ est calculé par la fonction `plus` et $A^*$ est calculé par la fonction `star`. La solution de l'équation est donnée par la fonction `astarb`. Une solution existe dès que le plus grand circuit dans le graphe
d'adjacence de la matrice a un poids negatif ou nul. Le poids d'un circuit étant donné par la
somme des poids des arcs (coefficients de la matrice) le constituant.

In [134]:
star(MP(2))

(max,+) Inf

In [135]:
star(MP(-2))

(max,+) 0

In [136]:
A = MP([-3 -2; -1 0])
plus(A)

2×2 (max,+) dense matrix:
   0   -2
  -1    0


In [137]:
star(A)

2×2 (max,+) dense matrix:
   0   -2
  -1    0


In [138]:
plus(A) == A * star(A)

true

In [139]:
star(zeros(MP, 2,2)) == star(full(zeros(MP, 2,2)))

true

In [140]:
b = MP([mp0; mp1])

2-element (max,+) vector:
  .
  0


In [141]:
x = astarb(A, b)

2-element (max,+) vector:
  -2
   0


In [142]:
x == A * x + b

true